**RETO DATATON BANCOLOMBIA**

**GRUPO:** UN_Data

**INTEGRANTES:**

*   Jhon Daniel Hoyos Arias.
*   Ivan Santiago Rojas Martinez.
*   Yised Katerine Naranjo García.

# **Avance**


*   **Metodología**







1.   La metodologia utilizada para el clasificador de noticias fue  a partir de ***Embeddings pre entrenados de palabras (word2vec)*** Para la creación de un diccionario representativo de cada categoría de interés, que luego fue utilizado para determinar las frecuencias de ocurrencia de las mismas, para clasificar.

2.   La metodología utilizada para definir la participación del cliente o el sector en la noticia, fue a partir de las frecuencias del nombre o el sector en el texto, título y url de las noticias.


***¿Que son embeddings pre entrenados?***

se define como vectores de palabras (word2vec), y a estos se les llama embedding de palabras.
Word2Vec es un algoritmo con el cual podemos usar un vector para representar correctamente las palabras de una manera que captura las relaciones semánticas o de significado. Por ejemplo, al analizar estos vectores podemos saber si las palabras son similares u opuestas (analizando la dirección y el sentido del vector). Estos vectores representan el coseno del ángulo entre la diferencia de similitud dada entre 1 y -1 para un par de palabras.

Word2Vec es, además, un algoritmo que podemos utilizar para representar correctamente las palabras de manera que se capturara las relaciones semánticas o de significado, esto es importante, porque El uso de la semántica distribucional para entrenar modelos de lenguaje tiene una gran ventaja sobre la mayoría de los otros modelos de aprendizaje automático. Esa ventaja es que podemos entrenarlos con un texto corriente, que tenemos en abundancia, ya que la información que necesitamos es la frecuencia con las que las palabras suelen aparecer juntas.

***¿Por qué esta metodología ?***

Cuando hablamos de embeddings pre entrenados, hacemos referencia a un conjunto de vectores de números que representan el grado de asociación entre palabras que han sido evaluadas y ejecutadas en un gran número de textos. Gracias a la semántica que word2vec ofrece, para la clasificación de nuestras noticias utilizaremos un modelo de embeddings pre entrenados, de modo que crearemos un vocabulario único y con un alto grado de precisión para nuestras palabras clave, que representan nuestras categorías de clasificación.

*   **librerias utilizadas**

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk import FreqDist
import re
import collections
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

*   **Estrategias**

Tomamos un conjunto de 500.000 palabras embedding para entrenar nuestras noticias y de esta manera obtener un diccionario representativo de palabras asociadas a las categorías de interés para el banco, lo interesante es que nuestro modelo asocia la semántica del texto para calcular el nivel de asociación de cada categoría a cada palabra por lo que es bastante informativo.

Para definir si la participación del cliente o el sector era significativa dentro de la noticia no se utilizó una metodología muy diferente a la propuesta, le apostamos a una mejor depuración de las palabras.



*  **stoppers**


Uno de los limitantes principales es que desde la recolección de las noticias no se consideró la acentuación de las palabras, por lo que nuestro modelo pre entrenado pierde mucha calidad de asociación, sin embargo, pudimos corregir de cierta manera el impacto que esto tenía sobre nuestras categorías de clasificación, asociando el acento a ciertas palabras.
 
Otra situación importante de consideración, es el tema de la participación del cliente o el sector en la noticia, no quedamos muy a gusto con la clasificación presentada, por lo que se continúa trabajando en una mejora mucho más óptima y precisa donde se considere las relaciones semánticas. 



*  **Resultados**

Se logró crear un clasificador de noticias y su respectiva participación, a partir de las frecuencias de aparición de las palabras asociadas a su respectiva categoría. 

# **CARGANDO LAS BASES DE DATOS**

Se cargan las tres bases de datos para la creación del sistema de recomendación de noticias con el fin de ofrecer a los clientes corporativo del banco información importante y confiable respecto a los sectores en lo que estos se encuentran.

In [ ]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

In [ ]:
db_clients = pd.read_csv("/content/drive/Shareddrives/Dataton_Bancolombia2022/Data /clientes.csv")
db_news = pd.read_csv("/content/drive/Shareddrives/Dataton_Bancolombia2022/Data /noticias.csv")
db_clients_news = pd.read_csv("/content/drive/Shareddrives/Dataton_Bancolombia2022/Data /clientes_noticias.csv")

# **DEPURACIÓN GENERAL**

In [ ]:
db_clients["desc_ciiu_division"] = db_clients["desc_ciiu_division"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))
db_clients["desc_ciiuu_clase"] = db_clients["desc_ciiuu_clase"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))
db_clients["desc_ciuu_grupo"] = db_clients["desc_ciuu_grupo"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))
db_clients["subsec"] = db_clients["subsec"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))

In [ ]:
db_news["news_title"] = db_news["news_title"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))
db_news["news_text_content"] = db_news["news_text_content"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))


In [ ]:
db_clients_news.head()

,nit,news_id,news_url_absolute,news_init_date,news_final_date
0,900378212,news10006,https://www.bluradio.com/economia/precio-dolar...,2022-07-30,2022-08-14
1,900378212,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14
2,860034313,news10011,https://www.semana.com/economia/macroeconomia/...,2022-07-30,2022-08-14
3,900378212,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14
4,900166896,news10015,https://elcomercio.pe/respuestas/que/gustavo-p...,2022-07-30,2022-08-14


# **LIMPIEZA DE DATOS Y TOKENIZACIÓN**

Se realiza una limpieza de texto, en la base de datos, con el fin de eliminar todo aquello no este sumando información o patrones importantes para el análisis, mediante la función $limpiar tokenizar^1$  limpiando paso a paso y luego procediendo a una tokenización representando el texto como una lista de palabras. En ese sentido, dado que lo que interesa es realizar una buena categorización de noticias, se procede a eliminar:


*   Signos de puntuación.
*   Eliminación de patrones no informativos como páginas web (http)
* Eliminicación de números
* Eliminación de espacio en blanco adicionales
* Conversión del texto a minúsculas, para estandarización de las palabras)
* Eliminación de tokens con una longitud menor a 2, para encargarse de palabras que no son de mucha relevancia para el análisis como: "y", "o", entre otras. 



In [ ]:
 def clean_tokenizer_news(texto):

    nuevo_texto = texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 2]
    
    return(nuevo_texto)
    

In [ ]:
 def clean_tokenizer_url(texto):
    nuevo_texto = texto.lower()
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    nuevo_texto = re.sub("https?", ' ', nuevo_texto)
    nuevo_texto = re.sub("www?", ' ', nuevo_texto)
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    nuevo_texto = nuevo_texto.split(sep = '-')
    nuevo_texto = [token for token in nuevo_texto if len(token) >2] 
    return(nuevo_texto)

In [ ]:
textUrl=[]
for i in range(0, db_news.shape[0]):
  textUrl.append(clean_tokenizer_url(db_news.news_url_absolute[i]))

textTitle=[]
for i in range(1, db_news.shape[0]):
  textTitle.append(clean_tokenizer_news(db_news.news_title[i]))

textNews=[]
for i in range(1, db_news.shape[0]):
  textNews.append(clean_tokenizer_news(db_news.news_text_content[i]))


# **ACENTUACIÓN DE LAS PALABRAS CLAVES**
Se decidió hacer la acentuación de palabras claves dentro de nuestro modelo, dado que los embeddings pre entrenados funcionan mejor con palabras acentuadas, de esta manera se soluciona un poco el efecto de no considerar los acentos, y se logra una mejor precisión en el ajuste de los sentimientos de nuestras palabras.  


In [ ]:
for title in textTitle:
    for i in range(len(title)):
        if title[i] == "macroeconomia":
            title[i] = "macroeconomía"
        
        if title[i] == "reputacion":
           title[i] = "reputación"

        if title[i] == "innovacion":
           title[i] = "innovación"

        if title[i] == "economía":
           title[i] = "economía"
                    
        
for new in textNews:
    for i in range(len(new)):
        if new[i] == "macroeconomia":
           new[i] = "macroeconomía"
        
        if new[i] == "reputacion":
           new[i] = "reputación"
        
        if new[i] == "innovacion":
           new[i] = "innovación"

        if new[i] == "economía":
           new[i] = "economía"

for url in textUrl:
    for i in range(len(url)):
        if url[i] == "macroeconomia":
           url[i] = "macroeconomía"

        if url[i] == "reputacion":
           url[i] = "reputación"
        
        if url[i] == "innovacion":
           url[i] = "innovación"

        if url[i] == "economía":
           url[i] = "economía"


# **MODELO PRE-ENTRENADO**


Para la categorización de las noticias, inicialmente se hace uso de un modelo Enbedding-s pre-entrenado, ya que este permite la representación de texto, mediante la incorporación de algoritmos con Wordvec2 y métodos de entrenamiento aproximados. Este modelo nos permite en nuestro análisis que mediante esta incrustación de palabras con un enfoque que permita capture algo acerca del significado de las palabras y lograr con esto una mejor clasificación de las noticias. Estas incrustaciones de palabras funcionan utilizando el algoritmo wordvec2 para entrenar un conjunto de vectores densos y continuos de longitud fija basados en un gran corpus de texto, en donde aprender una palabra incrustada a partir de un texto implica cargar y organizar el texto en frases y proporcionarlas al constructor de una nueva instancia, cada frase debe ser simbólica para que pueda categorizarse adecuadamente.
 
• El siguiente código descarga un archivo que contiene 1.000.653 incrustaciones de palabras con una dimensión de 300 entrenadas, estas Enbeddings de palabras se entrenaron con un algoritmo modelo skip-gram con muestreo negativo, con una frecuencia mínima de 5 palabras y además las 273 palabras más comunes sé re-muestrearon. Luego de esto se obtiene con este modelo un total de, 771508817 palabras sin procesar y un total de 1000653 tokens únicos para la categorización.


In [ ]:
!wget https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2
!bzip2 -d SBW-vectors-300-min5.txt.bz2

--2022-11-01 00:09:00--  https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2
Resolving cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)... 200.16.17.55
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 818175453 (780M) [application/x-bzip2]
Saving to: ‘SBW-vectors-300-min5.txt.bz2’

SBW-vectors-300-min 100%[===================>] 780.27M  18.4MB/s    in 44s     

2022-11-01 00:09:45 (17.9 MB/s) - ‘SBW-vectors-300-min5.txt.bz2’ saved [818175453/818175453]



# **CANTIDAD DE EMBEDDING Y DIMENSIÓN**

Se observa la cantidad de cada Embedding y la dimensión de cada uno de ellos. Luego, cada línea contiene el token vectorizado y luego el vector propiamente dicho.


In [ ]:
with open( "SBW-vectors-300-min5.txt", 'r') as f:
  n_lin = 0
  for line in f:
    print(line)
    n_lin += 1
    if n_lin>3: break

1000653 300

de -0.029648 0.011336 0.019949 -0.088832 -0.025225 0.056844 0.025473 0.014068 0.163694 -0.067154 0.014738 0.027134 0.066443 -0.044846 -0.044987 -0.040898 0.030311 0.034196 -0.049240 0.008537 -0.068091 -0.087938 0.035300 0.149385 -0.012350 0.012613 0.029350 0.069596 0.039111 0.057652 0.069954 -0.066217 -0.041784 0.028623 0.026772 -0.066392 0.002953 -0.012188 -0.030363 0.040222 0.034858 0.027469 -0.029034 -0.048748 -0.038582 -0.051553 -0.033501 -0.019008 0.003043 0.110712 -0.025096 0.111082 0.035244 0.114207 0.010195 0.051511 -0.040649 -0.113944 0.044873 0.052011 0.067360 0.049054 -0.127085 -0.031846 0.032848 0.040825 -0.084873 0.059801 -0.067424 0.016531 -0.084565 0.057024 0.083288 -0.010136 -0.048508 0.051757 0.046664 0.018102 -0.052320 -0.000765 0.053662 -0.009967 0.082858 0.009068 0.054575 -0.003466 -0.023376 0.023069 0.088513 0.018504 -0.039503 -0.032980 -0.002139 0.000010 -0.107627 0.007699 0.046351 -0.003062 0.030500 0.113650 0.032536 -0.097301 -0.013734 0.098345 0.08

# **CARGANDO EL CONTENIDO EN LA MEMORIA**

Se generá una clase que permite almacenar los enbeddings en la memoria para poder acceder a ellos de manera más ordenada.

Esta clase contendrá varios atributos útiles:

*   idx_to_token: es una lista que contendrá los tokens. 
*   idx_to_vec: es una lista que contendrá los embeddings. 
*   dim: es la dimensión de los embeddings.
*   token_to_idx: devuelve el id correspondiente al token pasado como parámetro.

In [ ]:
import torch
class TokenEmbedding:
  """Token Embedding."""
  def __init__(self, file_name, n):
    self.idx_to_token, self.idx_to_vec, self.dim = self._load_embedding(
        file_name, n)
    self.unknown_idx = 0
    self.token_to_idx = {token: idx for idx, token in
                          enumerate(self.idx_to_token)}
  

  def _load_embedding(self, file_name, n):
    idx_to_token, idx_to_vec = ['<unk>'], []
    with open( file_name, 'r') as f:
      first_read = True
      i=0
      for line in f:
        if n<i: break
        else: i+=1
        if first_read:
          first_read = False
          continue
        elems = line.rstrip().split(' ')
        token, elems = elems[0], [float(elem) for elem in elems[1:]]
        # Skip header information, such as the top row in fastText
        if len(elems) > 1:
            idx_to_token.append(token)
            idx_to_vec.append(elems)
    idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
    return idx_to_token, torch.tensor(idx_to_vec), len(idx_to_vec[0])

  def __getitem__(self, tokens):
    indices = [self.token_to_idx.get(token, self.unknown_idx)
                for token in tokens]
    vecs = self.idx_to_vec[torch.tensor(indices)]
    return vecs

  def __len__(self):
    return len(self.idx_to_token)

Se procede a cargar los embeddings en un objeto llamado  *spanish_w2v*. Donde se cargaron 500000 tokens

In [ ]:
spanish_w2v = TokenEmbedding("SBW-vectors-300-min5.txt",500000)

# **PRUEBA DE DATOS PRE-ENTRENADOS**

Se evaluarán lo embeddings usando los vectores del algoritmo *wordvec2* para encontrar palabras similares respecto a cada clasificación de las noticias.


In [ ]:
id_economia = spanish_w2v.token_to_idx["reputacion"]
spanish_w2v.idx_to_vec[id_economia]

# **IMPLEMENTACIÓN DE FUNCIÓN KNN PARA SIMILITUD DE PALABRAS**
Se implementa la siguiente función de K vecinos más cercanos KNN, para encontrar palabras similares, usando la similitud de coseno de los vectores de palabras del modelo entrenado.

In [ ]:
def knn(W, x, k):
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

Con el siguiente código se buscan las palabras similares de la clasificación que de sea de las noticias, utilizando las incrustaciones de palabras pre-entrenadas.

In [ ]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Exclude the input word
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

get_similar_tokens('innovación', 100, spanish_w2v)

# **ENTRENAMIENTO DE NOTICIAS**

En este apartado, se ingresan vectores de la palabra tokenizadas referentes al texto de las noticias, y se obtiene un vocabulario (vocab_news) que tiene la misma estructura de los enbedding pre-entrenados, con un tamaño de 65173.
 
Cabe notar que los embeddings pre-entrenados que estamos tomando tienen una capacidad de un millón de palabras, pero solo se toman 500.000 por memoria computacional.




In [ ]:
def make_vocab(oraciones,min_freq=1):
  #Comprueba que oraciones es una lista de listas
  if oraciones and isinstance(oraciones[0], list):
    #Transforma una lista anidada en una lista simple 
    tokens = [token for line in oraciones for token in line]
  counter_obj = collections.Counter()
  counter_obj.update(tokens)
  sorted_by_freq_tuples = sorted(counter_obj.items(), key=lambda x: x[1], reverse=True)
  ordered_dict = OrderedDict(sorted_by_freq_tuples)
  vocabulario = vocab(ordered_dict, min_freq=min_freq)
  return vocabulario, ordered_dict
vocab_news, ordered_dict = make_vocab(textNews,5)


In [ ]:
f"Este vocabulario tiene un tamaño de {len(vocab_news.get_itos())}"

'Este vocabulario tiene un tamaño de 65173'

# **CANTIDAD DE EMBEDDING Y DIMENSIÓN PARA EL ENTRENAMIENTO DE NOTICIAS**



Este apartado es utilizado para crear una capa de embeddings, y son almacenados en una matriz, la cual contendrá un vector de números similares a los tensores obtenidos de los embeddings pre entrenados.

In [ ]:
matrix_len = len(vocab_news.get_itos())
weights_matrix = np.zeros((len(vocab_news.get_itos()), spanish_w2v.dim))
words_found = 0
words_not_found = []

for i, word in enumerate(vocab_news.get_itos()):
    try: 
        weights_matrix[i] = spanish_w2v[[word]][0]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(spanish_w2v.dim, ))
        words_not_found.append(word)

weights_matrix.shape

(65173, 300)

# **CREANDO UNA RED NEURONAL**
Para esta parte, se hace la transformación de la matriz que contiene los vectores de números asociados, y se transforma a los tensores necesarios para crear los embeddings de nuestras noticias.
Por último, se prueba con un modelo de 256 variables ocultas y 2 capas de embeddings con el fin de no sobre ajustar el modelo.


In [ ]:

from torch import nn 

def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': torch.tensor(weights_matrix)})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [ ]:

class ToyNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super().__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)

In [ ]:

model = ToyNN(weights_matrix,256,2)

# **Validando nuestro modelo**

In [ ]:
def get_similar_tokens2(query_token, k, embed):
    W = embed.weight.data
    assert vocab_news.__contains__(query_token), "El token no está en el vocabulario"
    x = W[vocab_news[query_token]]
    # Compute the cosine similarity. Add 1e-9 for numerical stability
    cos = torch.mv(W, x) / torch.sqrt(torch.sum(W * W, dim=1) *
                                      torch.sum(x * x) + 1e-9)
    topk = torch.topk(cos, k=k+1)[1].cpu().numpy().astype('int32')
    tokens = vocab_news.get_itos()
    arrayText=[]
    for i in topk[1:]:  # Remove the input words
        #print(f'cosine sim={float(cos[i]):.3f}: {tokens[i]}')
        arrayText.append(tokens[i])

    return(arrayText)


In [ ]:
k=get_similar_tokens2('macroeconomía', 50, model.embedding)
k

# **CREACIÓN DEL DICCIONARIO**

Se crea un diccionario para almacenar las palabras con datos claves para la respectiva clasificación de las palabras según las diferentes categorías que se estan teniendo en cuenta para la realización del recomendador de noticias:

* Macroeconomía.
* Sostenibilidad
* Innovación
* Regulaciones
* Alianzas
* Reputación

In [ ]:
categorys = ["macroeconomía", "sostenibilidad", "innovación", "regulaciones", "alianzas", "reputación"]
diccionary={}

for category in categorys:
    k = get_similar_tokens2(category, 50, model.embedding)
    diccionary[category] = k


print(diccionary)


{'macroeconomía': ['economistas', 'keynesiana', 'finanzas', 'keynesianismo', 'economista', 'monetaria', 'monetarias', 'ciencias', 'recesiones', 'marginalista', 'financiera', 'inflacionarios', 'contractiva', 'sostenibilidad', 'competitividad', 'gobernanza', 'neoliberales', 'dolarizar', 'coyuntura', 'monetario', 'distributivos', 'coyunturales', 'neurociencia', 'monetarios', 'neoliberal', 'cientistas', 'transformacional', 'agronegocios', 'redistributivas', 'economics', 'inflaciones', 'fiscalidad', 'finanza', 'sustentabilidad', 'neoliberalismo', 'inflacionarias', 'extractivista', 'ambientalismo', 'prospectiva', 'humanidades', 'microfinanzas', 'transdisciplinar', 'coyuntural', 'externalidades', 'especulativa', 'emprendedurismo', 'inflacionario', 'cambiaria', 'bancarizada', 'dolarizada'], 'sostenibilidad': ['sustentabilidad', 'sostenible', 'viabilidad', 'sostenibles', 'competitividad', 'medioambiental', 'sustentable', 'eficiencia', 'ambiental', 'medioambientales', 'bienestar', 'gobernanza', 

# **CLASIFICACIÓN SEGÚN CATEGORÍA DE LAS PALABRAS**

Primeramente se identifica la cantidad de palabras según la categoría de la noticia con un conteó de frecuencias y se procede filtrar la información por titulo, url y texto.

In [ ]:
data_count_categorys=db_news.copy()
data_count_categorys[['news_text_content']]=data_count_categorys[['news_text_content']].astype(str)
data_count_categorys['news_url_text']=pd.DataFrame(textUrl)

In [ ]:
data_count_categorys2=data_count_categorys.copy()
for i in diccionary:
  exp_regular='\\b'+'(s)?(es)?\\b|\\b'.join(list(diccionary[i]))+'(s)?(es)?\\b'
  nomColum1='count_'+i+'_title'
  nomColum2='count_'+i+'_text'
  nomColum3='count_'+i+'_url'

  data_count_categorys2[nomColum1]=data_count_categorys2['news_title'].apply(lambda x: len(re.findall(exp_regular,x,re.IGNORECASE)))
  data_count_categorys2[nomColum2]=data_count_categorys2['news_text_content'].apply(lambda x: len(re.findall(exp_regular,x,re.IGNORECASE)))
  data_count_categorys2[nomColum3]=data_count_categorys2['news_url_text'].apply(lambda x: len(re.findall(exp_regular,x,re.IGNORECASE)))


In [ ]:
columnas2 = ['news_id',
       'count_macroeconomía_title', 'count_macroeconomía_text','count_macroeconomía_url',
       'count_sostenibilidad_title','count_sostenibilidad_text','count_sostenibilidad_url',
       'count_innovación_title', 'count_innovación_text', 'count_innovación_url',
       'count_regulaciones_title','count_regulaciones_text','count_regulaciones_url',
       'count_alianzas_title','count_alianzas_text', 'count_alianzas_url',
       'count_reputación_title','count_reputación_text','count_reputación_url']

In [ ]:
data_count_categorys2[columnas2]

# **CLASIFICACIÓN SEGÚN EL CLIENTE CORPORATIVO O SECTOR**

Primero se realiza una eliminación de palaras que no son importantes en el nombre del cliente tales como: "ssa", "s.a", entre otros.

In [ ]:
db_clients["nombre"] = db_clients["nombre"].apply(lambda x: re.sub("\\ssa\\b|\\sltda?\\b|\\ s a\\b|\\ sas\\b|\\ s.a.\\b|\\ 2 e p\\b|\\ ltda\\b|\\ c.i \\b|\\.\\b|\\ esp\\b", "",x.lower()))
db_clients["nombre"] = db_clients["nombre"].apply(lambda x: re.sub(r'[^\w\s]', "",x.lower()))

In [ ]:
data_result = pd.DataFrame()

column_select = ['nit', 'news_id', 'news_url_absolute', 'news_init_date', 
    'news_final_date', 'news_title', 'count_name_tittle',
    'count_name_text','count_name_url', 'count_sect_tittle', 'count_sect_text','count_sect_url','count_ciiu_tittle', 'count_ciiu_text','count_ciiu_url'
       ]

for j, i in enumerate(db_clients["nit"]):
    clie_news_filter = db_clients_news[db_clients_news["nit"]==i][["news_id", "nit"]]
    data_filter = pd.merge(data_count_categorys, clie_news_filter, on= "news_id")
    nombre = db_clients[db_clients["nit"]==i]["nombre"].iloc[0]
    sector = db_clients[db_clients["nit"]==i]["desc_ciiu_division"].iloc[0]
    ciiu = db_clients[db_clients["nit"]==i]["subsec"].iloc[0]

    data_filter["count_name_tittle"] = data_filter["news_title"].apply(lambda x: len(re.findall(nombre, x , re.IGNORECASE)))
    data_filter["count_name_text"] = data_filter["news_text_content"].apply(lambda x: len(re.findall(nombre, x, re.IGNORECASE)))
    data_filter["count_name_url"] = data_filter["news_url_text"].apply(lambda x: len(re.findall(nombre, x, re.IGNORECASE)))
    data_filter["count_sect_tittle"] = data_filter["news_title"].apply(lambda x: len(re.findall(sector, x, re.IGNORECASE)))
    data_filter["count_sect_text"] = data_filter["news_text_content"].apply(lambda x: len(re.findall(sector, x, re.IGNORECASE)))
    data_filter["count_sect_url"] = data_filter["news_url_text"].apply(lambda x: len(re.findall(sector, x, re.IGNORECASE)))
    data_filter["count_ciiu_tittle"] = data_filter["news_title"].apply(lambda x: len(re.findall(ciiu, x, re.IGNORECASE)))
    data_filter["count_ciiu_text"] = data_filter["news_text_content"].apply(lambda x: len(re.findall(ciiu, x, re.IGNORECASE)))
    data_filter["count_ciiu_url"] = data_filter["news_url_text"].apply(lambda x: len(re.findall(ciiu, x, re.IGNORECASE)))
    
    data_result = pd.concat([data_result,data_filter[column_select]], ignore_index=True)
    print("Termine de contar el cliente:" + str(i) + " avance " + str((j + 1)/len(db_clients["nit"])))

In [ ]:
data_result2 = pd.merge(db_clients_news[["nit", "news_id"]], data_count_categorys2[columnas2], on = "news_id", how = "left")
data_result_final_aux = pd.merge(data_result, data_result2, on = ["nit", "news_id"])
data_result_final = pd.merge(db_clients[["nit", "nombre"]], data_result_final_aux, on = "nit")
data_result_final.head()

In [ ]:
data_result_final

In [ ]:
nit=[]
id_new=[]
clasification=[]
particip=[]
for i in range(0, data_result_final.shape[0]):
    id_new.append(data_result_final.news_id[i])
    nit.append(data_result_final.nit[i])
    sum_macroeconomia, sum_sostenibilidad, sum_innovacion, sum_regulacion, sum_alianzas,  sum_reputacion = 0, 0, 0, 0, 0, 0
    count=[]
    sum_macroeconomia=data_result_final.count_macroeconomía_text[i]+2*data_result_final.count_macroeconomía_title[i]+4*data_result_final.count_macroeconomía_url[i]          
    sum_sostenibilidad=data_result_final.count_sostenibilidad_text[i]+2*data_result_final.count_sostenibilidad_title[i]+4*data_result_final.count_sostenibilidad_url[i]
    sum_innovacion=data_result_final.count_innovación_text[i]+2*data_result_final.count_innovación_title[i]+4*data_result_final.count_innovación_url[i]
    sum_regulacion=data_result_final.count_regulaciones_text[i]+2*data_result_final.count_regulaciones_title[i]+data_result_final.count_regulaciones_url[i]
    sum_alianzas=data_result_final.count_alianzas_text[i]+2*data_result_final.count_alianzas_title[i]+4*data_result_final.count_alianzas_url[i]
    sum_reputacion=data_result_final.count_reputación_text[i]+2*data_result_final.count_reputación_title[i]+4*data_result_final.count_reputación_url[i]
    
    count += [sum_macroeconomia, sum_sostenibilidad, sum_innovacion, sum_regulacion, sum_alianzas,  sum_reputacion]
    max_value = max(count) 
    index=count.index(max_value)
    
    if max_value==0:
      clasification.append('Descartable')
    else:
         if index==0:
            clasification.append('macroeconomía')              
         if index==1:
            clasification.append('sostenibilidad')
         if index==2:
            clasification.append('innovacion')
         if index==3:
            clasification.append('regulaciones')
         if index==4:
            clasification.append('alianzas')
         if index==5:
            clasification.append('reputacion')

    if data_result_final.count_name_tittle[i]>0 or data_result_final.count_name_text[i] >0 or data_result_final.count_name_url[i] >0 :       
            particip.append('Cliente')
    elif data_result_final.count_sect_tittle[i]>0 or data_result_final.count_sect_text[i] >0 or data_result_final.count_sect_url[i] >0 :       
             particip.append('Sector') 
    elif data_result_final.count_ciiu_tittle[i]>0 or data_result_final.count_ciiu_text[i]>0 or data_result_final.count_ciiu_url[i] >0 :       
             particip.append('Sector') 
    else:
        particip.append('No Aplica')  

    


# **Clasificación**

In [ ]:
db_clasifier2 = pd.DataFrame()
db_clasifier2['nit']=nit
db_clasifier2['id_news']=id_new
db_clasifier2['participation']=particip
db_clasifier2['clasification']=clasification
db_clasifier2

,nit,id_news,participation,clasification
0,805027024,news12584,No Aplica,innovacion
1,805027024,news12758,No Aplica,regulaciones
2,805027024,news13241,No Aplica,macroeconomía
3,805027024,news15048,No Aplica,sostenibilidad
4,805027024,news15611,No Aplica,innovacion
...,...,...,...,...
74612,891102723,news99569,No Aplica,alianzas
74613,891102723,news99573,No Aplica,innovacion
74614,891102723,news99574,No Aplica,reputacion
74615,891102723,news99576,No Aplica,alianzas


##limpieza, tomado de: https://github.com/institutohumai/cursos-python/tree/master/NLP/3_Embeddings
